# Importing Packages

In [ ]:
import numpy as np
import cv2
import glob
import pandas as pd
import os
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf

# Train Data Loading

In [ ]:
file_path = '/kaggle/input/aptos2019/train_1.csv'
df = pd.read_csv(file_path)

In [ ]:
folder_path = '/kaggle/input/aptos2019/train_images/train_images'
image_pattern = '*.png'
image_files = glob.glob(os.path.join(folder_path, image_pattern))

# Data exploration

In [ ]:
img = cv2.imread('/kaggle/input/aptos2019/train_images/train_images/1ae8c165fd53.png')
print(img.shape)
plt.imshow(img)

# Pre processing

In [ ]:
data = []
labels = []

image_size=(128,128)
for index, row in df.iterrows():
    image_path = f"/kaggle/input/aptos2019/train_images/train_images/{row['id_code']}.png"
    image = cv2.imread(image_path)    
    image = cv2.resize(image, image_size)
    data.append(image)
    labels.append(row['diagnosis'])

# Normalization

In [ ]:
X_train = np.array(data)
X_train = (X_train - np.min(X_train)) / (np.max(X_train) - np.min(X_train))
y_train = np.array(labels, dtype=np.int32)

print(X_train.shape)
print(y_train.shape)

In [ ]:
print(y_train)

# Pretrained Model loading

In [ ]:
from tensorflow.keras.applications import VGG16,MobileNetV2,InceptionV3,DenseNet121,ResNet50,ResNet101

In [ ]:
base_model = ResNet50(input_shape=(128,128,3), include_top=False, weights='imagenet')
for layer in base_model.layers:
    layer.trainable = False
    
base_model.summary()

# Adding top layer

In [ ]:
from tensorflow.keras.layers import Flatten,Dense

new_model=base_model.get_layer('conv5_block3_out').output #last layer of your pretrained model
output1=Flatten()(new_model)
output=Dense(units=512,activation='relu')(output1)
output=Dense(units=5, activation='softmax')(output)

In [ ]:
from tensorflow.keras.models import Model
model = Model(inputs=base_model.input, outputs=output)
model.summary()

# Validation Data

In [ ]:
val_df = pd.read_csv('/kaggle/input/aptos2019/valid.csv')

val_data = []
val_labels = []

image_size = (128,128)

for index, row in val_df.iterrows():
    image_path = f"/kaggle/input/aptos2019/val_images/val_images/{row['id_code']}.png"
    image = cv2.imread(image_path)
    image = cv2.resize(image, image_size)
    val_data.append(image)
    val_labels.append(row['diagnosis'])

In [ ]:
X_val = np.array(val_data)
X_val = (X_val - np.min(X_val)) / (np.max(X_val) - np.min(X_val)).astype(np.float32)
y_val = np.array(val_labels, dtype=np.int32)

print(X_val.shape)
print(y_val.shape)

In [ ]:
print(y_val)

# Data Training

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history=model.fit(X_train,y_train,validation_data=(X_val,y_val),epochs=50, batch_size=16)

# Test Data

In [ ]:
test_df = pd.read_csv('/kaggle/input/aptos2019/test.csv')

test_data = []
test_labels = []

image_size = (128,128)

for index, row in test_df.iterrows():
    image_path = f"/kaggle/input/aptos2019/test_images/test_images/{row['id_code']}.png"
    image = cv2.imread(image_path)
    image = cv2.resize(image, image_size)
    test_data.append(image)
    test_labels.append(row['diagnosis'])
    
X_test = np.array(test_data)
X_test = (X_test - np.min(X_test)) / (np.max(X_test) - np.min(X_test)).astype(np.float32)
y_test = np.array(test_labels, dtype=np.int32)

print(X_test.shape)
print(y_test.shape)

# Model Evaluation

In [ ]:
loss, accuracy = model.evaluate(X_test,y_test)

print(f"Test loss: {loss:.4f}")
print(f"Test accuracy: {accuracy:.4f}")

# Reports & Confusion Matrix

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# Convert true labels to one-hot encoded format
y_test_oh = tf.keras.utils.to_categorical(y_test, num_classes=5)

# Get the true labels and predicted probabilities for the validation data
predictions = model.predict(X_test)
predicted_classes = np.argmax(predictions, axis=1)

# Generate the classification report
class_names = ['No DR', 'Mild DR', 'Moderate DR', 'Severe DR', 'Proliferative DR']
report = classification_report(np.argmax(y_test_oh, axis=1), predicted_classes, target_names=class_names)
print("Classification Report:\n", report)

# Generate the confusion matrix
cm = confusion_matrix(np.argmax(y_test_oh, axis=1), predicted_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
import tensorflow.keras as keras

# Assuming you have already trained your model and have a trained model object named 'model'

# Save the model using save_model
keras.models.save_model(model, 'efficientnetb4.h5')

# Image Number in Different Class


In [ ]:
# train data

import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf

# Convert true labels to one-hot encoded format
y_train_onehot = tf.keras.utils.to_categorical(y_train, num_classes=5)

# Plot the distribution of images in different classes for the training set
plt.figure(figsize=(8, 6))
sns.countplot(x=np.argmax(y_train_onehot, axis=1), palette="viridis")
plt.xlabel('Class')
plt.ylabel('Number of Images')
plt.title('Distribution of Images in Different Classes (Training Set)')
plt.xticks(ticks=range(5), labels=['No DR', 'Mild DR', 'Moderate DR', 'Severe DR', 'Proliferative DR'])
plt.show()

In [ ]:
#val data

y_val_onehot = tf.keras.utils.to_categorical(y_val, num_classes=5)

plt.figure(figsize=(8, 6))
sns.countplot(x=np.argmax(y_val_onehot, axis=1), palette="viridis")
plt.xlabel('Class')
plt.ylabel('Number of Images')
plt.title('Distribution of Images in Different Classes (Validation Set)')
plt.xticks(ticks=range(5), labels=['No DR', 'Mild DR', 'Moderate DR', 'Severe DR', 'Proliferative DR'])
plt.show()

In [ ]:
#test data

y_test_onehot = tf.keras.utils.to_categorical(y_test, num_classes=5)

plt.figure(figsize=(8, 6))
sns.countplot(x=np.argmax(y_test_onehot, axis=1), palette="viridis")
plt.xlabel('Class')
plt.ylabel('Number of Images')
plt.title('Distribution of Images in Different Classes (Test Set)')
plt.xticks(ticks=range(5), labels=['No DR', 'Mild DR', 'Moderate DR', 'Severe DR', 'Proliferative DR'])
plt.show()

# Implementing EfficientnetB0 Model

In [ ]:
import pandas as pd
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

# Load the training data
train_file_path = '/kaggle/input/aptos2019/train_1.csv'
train_df = pd.read_csv(train_file_path)
train_folder_path = '/kaggle/input/aptos2019/train_images/train_images'

# Preprocess training data
train_data = []
train_labels = []
image_size = (224, 224)

for index, row in train_df.iterrows():
    image_path = os.path.join(train_folder_path, f"{row['id_code']}.png")
    image = cv2.imread(image_path)
    image = cv2.resize(image, image_size)
    train_data.append(image)
    train_labels.append(row['diagnosis'])

# Normalize and one-hot encode training data
X_train = np.array(train_data)
X_train = (X_train - np.min(X_train)) / (np.max(X_train) - np.min(X_train))
y_train = np.array(train_labels, dtype=np.int32)
X_train = np.reshape(X_train, (-1, 224, 224, 3))
y_train = to_categorical(y_train)

# Load the EfficientNetB0 model with pre-trained ImageNet weights, excluding the top (classification) layers
base_model = EfficientNetB0(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Freeze all base model layers to prevent them from being updated during training
for layer in base_model.layers:
    layer.trainable = False

# Define additional layers for classification
new_model = base_model.output
new_model = Flatten()(new_model)
new_model = Dropout(0.2)(new_model)
new_model = Dense(512, activation='relu')(new_model)
new_model = Dropout(0.2)(new_model)
output = Dense(5, activation='softmax')(new_model)

# Create the final model by combining the base model and the new layers
model_efficientnetB0 = Model(inputs=base_model.input, outputs=output)

# Compile the model
model_efficientnetB0.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define Early Stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model with fine-tuning
history_b0 = model_efficientnetB0.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2, callbacks=[early_stopping])

# Unfreeze some layers of the base model for fine-tuning
for layer in model_efficientnetB0.layers[-20:]:
    layer.trainable = True

# Lower the learning rate for fine-tuning
model_efficientnetB0.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), metrics=['accuracy'])

# Continue training the model with fine-tuning
history_fine_tuning = model_efficientnetB0.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2, callbacks=[early_stopping])


In [ ]:
# Test Data
test_df = pd.read_csv('/kaggle/input/aptos2019/test.csv')

test_data = []
test_labels = []

image_size = (224,224)

for index, row in test_df.iterrows():
    image_path = f"/kaggle/input/aptos2019/test_images/test_images/{row['id_code']}.png"
    image = cv2.imread(image_path)
    image = cv2.resize(image, image_size)
    test_data.append(image)
    test_labels.append(row['diagnosis'])
    
X_test = np.array(test_data)
X_test = (X_test - np.min(X_test)) / (np.max(X_test) - np.min(X_test)).astype(np.float32)
X_test = np.reshape(X_test, (-1, 224, 224, 3))
y_test = np.array(test_labels, dtype=np.int32)
y_test = to_categorical(y_test)

# Model Evaluation
loss, accuracy = model_efficientnetB0.evaluate(X_test,y_test)

print(f"Test loss: {loss:.4f}")
print(f"Test accuracy: {accuracy:.4f}")

# Implementing ResNet152V2 Model

In [ ]:
import pandas as pd
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model

# Load the training data
train_file_path = '/kaggle/input/aptos2019/train_1.csv'
train_df = pd.read_csv(train_file_path)
train_folder_path = '/kaggle/input/aptos2019/train_images/train_images'

# Preprocess training data
train_data = []
train_labels = []
image_size = (380, 380)

for index, row in train_df.iterrows():
    image_path = os.path.join(train_folder_path, f"{row['id_code']}.png")
    image = cv2.imread(image_path)
    image = cv2.resize(image, image_size)
    train_data.append(image)
    train_labels.append(row['diagnosis'])

# Normalize and one-hot encode training data
X_train = np.array(train_data)
X_train = X_train.astype('float32') / 255.0  # Normalize pixel values to [0, 1]
y_train = np.array(train_labels, dtype=np.int32)
y_train = to_categorical(y_train)

# Load the validation data
val_file_path = '/kaggle/input/aptos2019/valid.csv'
val_df = pd.read_csv(val_file_path)
val_folder_path = '/kaggle/input/aptos2019/val_images/val_images'

# Preprocess validation data
val_data = []
val_labels = []
for index, row in val_df.iterrows():
    image_path = os.path.join(val_folder_path, f"{row['id_code']}.png")
    image = cv2.imread(image_path)
    image = cv2.resize(image, image_size)
    val_data.append(image)
    val_labels.append(row['diagnosis'])

# Normalize and one-hot encode validation data
X_val = np.array(val_data)
X_val = X_val.astype('float32') / 255.0  # Normalize pixel values to [0, 1]
y_val = np.array(val_labels, dtype=np.int32)
y_val = to_categorical(y_val)

# Load the ResNet152V2 model with pre-trained ImageNet weights
base_model = ResNet152V2(input_shape=(380, 380, 3), include_top=False, weights='imagenet')

# Freeze all base model layers to prevent them from being updated during training
for layer in base_model.layers:
    layer.trainable = False

# Define additional layers for classification
new_model = base_model.output
new_model = Flatten()(new_model)
new_model = Dropout(0.5)(new_model)
new_model = Dense(512, activation='relu')(new_model)
new_model = Dropout(0.5)(new_model)
output = Dense(5, activation='softmax')(new_model)

# Create the final model by combining the base model and the new layers
model_ResNet152V2 = Model(inputs=base_model.input, outputs=output)

# Compile the model
model_ResNet152V2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history_ResNet152V2 = model_ResNet152V2.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=16)


In [ ]:
# Test Data
test_df = pd.read_csv('/kaggle/input/aptos2019/test.csv')

test_data = []
test_labels = []

image_size = (380, 380)  # Adjusted to match the input size for ResNeXt50

for index, row in test_df.iterrows():
    image_path = f"/kaggle/input/aptos2019/test_images/test_images/{row['id_code']}.png"
    image = cv2.imread(image_path)
    image = cv2.resize(image, image_size)
    test_data.append(image)
    test_labels.append(row['diagnosis'])
    
X_test = np.array(test_data)
X_test = X_test.astype('float32') / 255.0  # Normalize pixel values to [0, 1] for ResNeXt50
X_test = np.reshape(X_test, (-1, 380, 380, 3))
y_test = np.array(test_labels, dtype=np.int32)
y_test = to_categorical(y_test)

# Model Evaluation
loss, accuracy = model_ResNet152V2.evaluate(X_test, y_test)

print(f"Test loss: {loss:.4f}")
print(f"Test accuracy: {accuracy:.4f}")


In [ ]:
import tensorflow.keras as keras

# Assuming you have already trained your model and have a trained model object named 'model'

# Save the model using save_model
keras.models.save_model(model_ResNet152V2, 'ResNet152V2.h5')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Predict labels for the test set
y_pred = model_ResNet152V2.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Compute confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred_classes)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.xticks(ticks=np.arange(5) + 0.5, labels=[0, 1, 2, 3, 4])
plt.yticks(ticks=np.arange(5) + 0.5, labels=[0, 1, 2, 3, 4])
plt.show()


In [ ]:

# Test Data
test_df = pd.read_csv('/kaggle/input/aptos2019/test.csv')

test_data = []
test_labels = []

image_size = (380, 380)  # Change image resolution to 380x380

for index, row in test_df.iterrows():
    image_path = f"/kaggle/input/aptos2019/test_images/test_images/{row['id_code']}.png"
    image = cv2.imread(image_path)
    image = cv2.resize(image, image_size)
    test_data.append(image)
    test_labels.append(row['diagnosis'])
    
X_test = np.array(test_data)
X_test = (X_test - np.min(X_test)) / (np.max(X_test) - np.min(X_test)).astype(np.float32)
X_test = np.reshape(X_test, (-1, 380, 380, 3))
y_test = np.array(test_labels, dtype=np.int32)
y_test = to_categorical(y_test)

# Model Evaluation
loss, accuracy = model_efficientnetB4.evaluate(X_test, y_test)

print(f"Test loss: {loss:.4f}")
print(f"Test accuracy: {accuracy:.4f}")


# Efficientnet B5 

In [ ]:
!pip install efficientnet

In [ ]:
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import class_weight
from keras.callbacks import LearningRateScheduler
import numpy as np
from keras.layers import Flatten, Dropout, Dense
from keras.models import Model
from efficientnet.tfkeras import EfficientNetB5  # Change import
import tensorflow as tf  # Import TensorFlow
from tensorflow.keras.activations import softmax  # Import softmax activation function

In [ ]:
# Load data
train = pd.read_csv('/kaggle/input/aptos2019/train_1.csv')
test = pd.read_csv('/kaggle/input/aptos2019/test.csv')
valid=pd.read_csv('/kaggle/input/aptos2019/valid.csv')

# Preprocess data
train["id_code"] = train["id_code"].apply(lambda x: x + ".png")
test["id_code"] = test["id_code"].apply(lambda x: x + ".png")
valid["id_code"] = valid["id_code"].apply(lambda x: x + ".png")
train['diagnosis'] = train['diagnosis'].astype('str')
test['diagnosis'] = test['diagnosis'].astype('str')
valid['diagnosis'] = valid['diagnosis'].astype('str')

# Define model parameters
BATCH_SIZE = 8
HEIGHT = 380
WIDTH = 380

# Create data generators
train_datagen=ImageDataGenerator(rescale=1./255)
train_generator=train_datagen.flow_from_dataframe(
    dataframe=train,
    directory="/kaggle/input/aptos2019/train_images/train_images",
    x_col="id_code",
    y_col="diagnosis",
    class_mode="categorical",
    batch_size=BATCH_SIZE,
    target_size=(HEIGHT, WIDTH),
    seed=0)

validation_datagen = ImageDataGenerator(rescale=1./255)
valid_generator=validation_datagen.flow_from_dataframe(
    dataframe=valid,
    directory="/kaggle/input/aptos2019/val_images/val_images",
    x_col="id_code",
    y_col="diagnosis",
    class_mode="categorical", 
    batch_size=BATCH_SIZE,   
    target_size=(HEIGHT, WIDTH),
    seed=0)

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(  
        dataframe=test,
        directory="/kaggle/input/aptos2019/test_images/test_images",
        x_col="id_code",
        y_col="diagnosis",  # Include y_col to get labels for evaluation
        batch_size=BATCH_SIZE,
        class_mode="categorical",  # Set class_mode to 'categorical'
        shuffle=False,
        target_size=(HEIGHT, WIDTH),
        seed=0)

In [ ]:
# Load the EfficientNetB5 model with pre-trained ImageNet weights
base_model = EfficientNetB5(input_shape=(HEIGHT, WIDTH, 3), include_top=False, weights='imagenet')

# Freeze all base model layers to prevent them from being updated during training
for layer in base_model.layers:
    layer.trainable = False

# Define additional layers for classification
x = base_model.output
x = Flatten()(x)
x = Dropout(0.2)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.2)(x)
output = Dense(5, activation=softmax)(x)  # Use softmax activation for multi-class classification

# Create the final model by combining the base model and the new layers
model_efficientnetB5 = Model(inputs=base_model.input, outputs=output)

# Compile the model
model_efficientnetB5.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define a learning rate schedule
def lr_schedule(epoch):
    lr = 0.001
    if epoch > 10:
        lr *= 0.5e-3
    elif epoch > 7:
        lr *= 1e-3
    elif epoch > 5:
        lr *= 1e-2
    elif epoch > 3:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

lr_scheduler = LearningRateScheduler(lr_schedule)

history_b5 = model_efficientnetB5.fit(
    train_generator,
    epochs=7,
    validation_data=valid_generator,
    callbacks=[lr_scheduler])  # use learning rate scheduler

In [ ]:
# Evaluate the model on test data
loss, accuracy = model_efficientnetB5.evaluate(test_generator, verbose=1)
print(f"Test loss: {loss:.4f}")
print(f"Test accuracy: {accuracy:.4f}")

In [ ]:
import tensorflow.keras as keras

# Assuming you have already trained your model and have a trained model object named 'model'

# Save the model using save_model
keras.models.save_model(model_efficientnetB5, 'efficientnetb5.h5')